# Inicialización

## Instrucciones

Adapta el código de preprocesamiento de las lecciones a la tarea de clasificación de reseñas:

Descarga el archivo con las reseñas /datasets/imdb_reviews_small.tsv.
Tokeniza cada reseña y considera que en el siguiente paso puedes tener hasta 512 tokens.
Encuentra la longitud mínima y máxima de los vectores después de la tokenización.
Aplica la técnica de relleno a los vectores y crea máscaras de atención para distinguir los tokens de los valores de relleno.

In [ ]:
import logging

import numpy as np
import pandas as pd

import torch
import transformers

# Cargar datos

Carga los datos de texto del archivo 'imdb_reviews_small.tsv' file.

Es un archivo de valores separados por tabuladores (TSV), lo cual significa que cada uno de los campos están separados por tabuladores (en lugar de comas, como has visto en otros ejercicios de TripleTen).

In [ ]:
data = # <tu código aquí>
#/datasets/imdb_reviews_small.tsv

# Tokenizador BERT

Crea el tokenizador BERT a partir de un modelo previamente entrenado que se llama 'bert-base-uncased' en transformadores. Puedes consultar rápidamente una descripción general [aquí](https://huggingface.co/transformers/pretrained_models.html), Y para obtener más detalles, puedes leer [aquí](https://huggingface.co/bert-base-uncased).

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Hay un ejemplo de cómo obtener tokens para un solo texto dado.

Puedes usarlo para procesar todos los datos que cargaste anteriormente. Como ya hay muchos textos, y es probable que los proceses en un bucle, las longitudes mínimas/máximas de los vectores se pueden calcular de dos formas: dentro de un bucle o después de un bucle.

En el último caso, los vectores de identificadores numéricos de tokens (`ids`) y máscaras de atención (`attention_mask`) se deben almacenar en dos listas separadas. Se pueden llamar `ids_list` y `atencion_mask_list`, respectivamente. El primer caso te permite evitar la creación de esas listas, a menos que desees utilizarlas con otra finalidad, por ejemplo, para propagarlas en un modelo BERT. No se requiere en esta tarea, pero se requerirá en el proyecto.

Teniendo en cuenta lo anterior, es posible que desees combinar ambas formas para calcular las longitudes mínimas/máximas de los vectores para tokens y máscaras de atención, y conservar el resultado del tokenizador para su posterior procesamiento. Solo considera que no tiene mucho sentido mantener vectores de más de 512 elementos, ya que esta es la longitud máxima de vectores que BERT puede aceptar.

In [ ]:
# textos a tokens
text = 'Es muy práctico utilizar transformadores'

# añadiendo este truco para suprimir avisos de salidas demasiado largas
# normalmente no necesitamos eso, pero en este caso queremos explorar
# cuál es la longitud máxima de ID para nuestro conjunto de reseñas
# por eso no truncamos la salida (ids) de max_length
# con los parámetros max_length=max_length y truncation=True
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

ids = tokenizer.encode(text.lower(), add_special_tokens=True)

# padding (agregar ceros al vector para hacer que su longitud sea igual a n)
n = 512
padded = np.array(ids[:n] + [0]*(n - len(ids)))

# creación de la máscara de atención para distinguir los tokens que nos interesan
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
print(ids)

[101, 2009, 2003, 2200, 18801, 2000, 2224, 19081, 102]


In [ ]:
print(padded)

[  101  2009  2003  2200 18801  2000  2224 19081   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [ ]:
print(attention_mask)

[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

Escribe tu código para tokenizar los datos de texto cargados.



In [ ]:
def tokenize_with_bert(texts):

    ids_list = []
    attention_mask_list = []

    # texto para las ID de tokens completadas con sus máscaras de atención
    min_tokenized_text_length = 1e7  # Comience con un valor grande
    max_tokenized_text_length = 0    # Empezar con 0

    # Iterar a través de cada texto en la lista de entrada
    # <tu código aquí>

      # Suprimir las advertencias sobre salidas prolongadas del tokenizador. esto es util
      # para explorar la distribución de longitudes de texto *sin* truncamiento, pero
      # debe eliminarse o comentarse para un uso normal cuando se desea truncarlo.
      logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

      # Tokenice el texto de entrada utilizando el tokenizador BERT.  Convertir a minúsculas
      # y agregue tokens especiales ([CLS] y [SEP]).
      ids = tokenizer.encode(# <tu código aquí>)

      # Calcule y actualice las longitudes mínimas y máximas de texto tokenizado.

      if ids_len
        # <tu código aquí>
      elif ids_len

      # Truncar los ID de los tokens si superan max_length.  Los modelos BERT tienen un límite
      # en la longitud de la secuencia de entrada.
      # <tu código aquí>

      # Rellene los ID de los tokens hasta max_length con 0.
      # <tu código aquí>

      # Crea una máscara de atención. 1 para tokens reales, 0 para tokens de relleno.

      # Agregue las identificaciones acolchadas y la máscara de atención a sus respectivas listas.
      ids_list.append(padded)
      attention_mask_list.append(attention_mask)

    print(f'La longitud mínima de los vectores: {min_tokenized_text_length}')
    print(f'La longitud máxima de los vectores: {max_tokenized_text_length}')

    return ids_list, attention_mask_list

Ejecuta el tokenizador para todos los datos. Puede tomar un poco de tiempo como

In [ ]:
ids_list, attention_mask_list = tokenize_with_bert(texts=data['review'])

Veamos algunos ejemplos.

In [ ]:
# cuenta elementos distintos de cero
ids_array = np.count_nonzero(np.array(ids_list), axis=1)
# almacenar el índice de la revisión que tiene la menor cantidad de tokens después de la tokenización
short_review_idx = np.argmin(np.count_nonzero(np.array(ids_list), axis=1))

In [ ]:
# la reseña más corta
data['review'].iloc[short_review_idx]

In [ ]:
# incorporación de la reseña más corta
ids_list[short_review_idx][:50]

In [ ]:
# Revisión más breve integrada basada en BERT
tokenizer.decode(ids_list[short_review_idx][:50], clean_up_tokenization_spaces=False)